# Example 03 - Official TorchSig Wideband Dataset
This notebook walks through how to use `torchsig` to generate the Official TorchSig Wideband Dataset.

-------------------------------------------

### Import Libraries

In [ ]:
from torchsig.utils.visualize import MaskClassVisualizer, mask_class_to_outline, complex_spectrogram_to_magnitude
from torchsig.transforms.target_transforms import DescToMaskClass, DescToListTuple
from torchsig.transforms import Spectrogram, Normalize
from torchsig.datasets.torchsig_wideband import TorchSigWideband
from torchsig.transforms.transforms import Compose
from torchsig.datasets import conf
from torchsig.datasets.datamodules import WidebandDataModule
from torchsig.datasets.signal_classes import torchsig_signals

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

-----------------------------
## Generate the Wideband TorchSig Wideband Dataset
To generate the TorchSigWideband dataset, several parameters are given to the imported `WidebandDataModule` class. These paramters are:
- `root` ~ A string to specify the root directory of where to generate and/or read an existing TorchSigWideband dataset
- `train` ~ A boolean to specify if the TorchSigWideband dataset should be the training (True) or validation (False) sets
- `qa` - A boolean to specify whether to generate a small subset of TorchSigNWideband (True), or the full dataset (False), default is True
- `impaired` ~ A boolean to specify if the TorchSigWideband dataset should be the clean version or the impaired version
- `transform` ~ Optionally, pass in any data transforms here if the dataset will be used in an ML training pipeline. Note: these transforms are not called during the dataset generation. The static saved dataset will always be in IQ format. The transform is only called when retrieving data examples.
- `target_transform` ~ Optionally, pass in any target transforms here if the dataset will be used in an ML training pipeline. Note: these target transforms are not called during the dataset generation. The static saved dataset will always be saved as tuples in the LMDB dataset. The target transform is only called when retrieving data examples.

A combination of the `train` and the `impaired` booleans determines which of the four (4) distinct TorchSigWideband datasets will be instantiated:
| `impaired` | `qa` | Result |
| ---------- | ---- | ------- |
| `False` | `False` | Clean datasets of train=250k examples and val=25k examples |
| `False` | `True` | Clean datasets of train=250 examples and val=250 examples |
| `True` | `False` | Impaired datasets of train=250k examples and val=25k examples |
| `True` | `True` | Impaired datasets of train=250 examples and val=250 examples |

The final option of the impaired validation set is the dataset to be used when reporting any results with the official TorchSigWideband dataset.

In [ ]:
# Generate TorchSigWideband DataModule
from torchsig.transforms import *
root = "./datasets/wideband"
impaired = True
qa = True
fft_size = 512
num_classes = len(torchsig_signals.class_list)
batch_size = 4
num_workers = 4

transform = Compose([                
    Spectrogram(nperseg=fft_size, noverlap=0, nfft=fft_size, mode='complex'),
    Normalize(norm=np.inf, flatten=True),
])

target_transform = Compose([
    DescToMaskClass(num_classes=num_classes, width=fft_size, height=fft_size),
])

datamodule = WidebandDataModule(
    root=root,
    impaired=impaired,
    qa=qa,
    fft_size=fft_size,
    num_classes=num_classes,
    transform=transform,
    target_transform=target_transform,
    batch_size=batch_size,
    num_workers=num_workers
)

datamodule.prepare_data()
datamodule.setup("fit")

wideband_train = datamodule.train

# Retrieve a sample and print out information
idx = np.random.randint(len(wideband_train))
data, label = wideband_train[idx]
print("Dataset length: {}".format(len(wideband_train)))
print("Data shape: {}".format(data.shape))
print("Label shape: {}".format(label.shape))

### Plot Subset to Verify
The `MaskClassVisualizer` can be passed a `Dataloader` and plot visualizations of the dataset. The `batch_size` of the `DataLoader` determines how many examples to plot for each iteration over the visualizer. Note that the dataset itself can be indexed and plotted sequentially using any familiar python plotting tools as an alternative plotting method to using the `spdata` `Visualizer` as shown below.

In [ ]:
data_loader = datamodule.train_dataloader()

visualizer = MaskClassVisualizer(
    data_loader=data_loader,
    visualize_transform=complex_spectrogram_to_magnitude,
    visualize_target_transform=mask_class_to_outline,
    class_list=torchsig_signals.class_list
)

for figure in iter(visualizer):
    figure.set_size_inches(16, 9)
    plt.show()
    break

----
### Analyze Dataset
The dataset can also be analyzed at the macro level for details such as the distribution of classes and number of signals per sample. The below analysis reads information directly from the non-target transformed tuple annotations. Since this is different than the above dataset instantiation, the dataset is re-instantiated for analysis.

In [ ]:
# Re-instantiate the TorchSigWideband Dataset witbatch_size=1, num_workers=1, hout a target transform and without using the RFData objects
datamodule = WidebandDataModule(
    root=root,
    impaired=impaired,
    qa=qa,
    fft_size=fft_size,
    num_classes=num_classes,
    transform=transform,
    target_transform=None,
    batch_size=1,
    num_workers=1
)
datamodule.prepare_data()
datamodule.setup("fit")

wideband_test = datamodule.train

# Loop through the dataset recording classes and SNRs
class_counter_dict = {
    class_name: 0 for class_name in list(wideband_test.class_list)
}
num_signals_per_sample = []

for idx in tqdm(range(len(wideband_test))):
    data, annotation = wideband_test[idx]
    num_signals_per_sample.append(len(annotation))
    for signal_annotation in annotation:
        class_counter_dict[signal_annotation["class_name"]] += 1

In [ ]:
# Plot the distribution of classes
class_names = list(class_counter_dict.keys())
num_classes = list(class_counter_dict.values())

plt.figure(figsize=(9,9))
plt.pie(num_classes, labels=class_names)
plt.title("Class Distribution Pie Chart")
plt.show()

plt.figure(figsize=(11,4))
plt.bar(class_names, num_classes)
plt.xticks(rotation=90)
plt.title("Class Distribution Bar Chart")
plt.xlabel("Modulation Class Name")
plt.ylabel("Counts")
plt.show()

The above distribution of classes shows all OFDM signals appearing less frequently than the remaining modulations. This makes sense because OFDM signals are drawn from a random distribution of bandwidths that are inherently larger than the remaining signals, meaning fewer OFDM signals can fit into a wideband spectrum without overlapping. Additionally, the random bursty probability and durations of OFDM signals makes it less likely to occupy a wideband capture with many short-time bursts, while the remaining modulations experience this behavior at a higher probility.

In [ ]:
# Plot the distribution of number of signals per sample
plt.figure(figsize=(11,8))
plt.hist(x=num_signals_per_sample, bins=np.arange(1,max(num_signals_per_sample)+1)-0.5)
plt.title("Distribution of Number of Signals Per Sample\nTotal Number: {} - Average: {} - Max: {}".format(
    sum(num_signals_per_sample),
    np.mean(np.asarray(num_signals_per_sample)),
    max(num_signals_per_sample),
))
plt.xlabel("Number of Signal Bins")
plt.ylabel("Counts")
plt.show()

The above distribution of the number of signals per sample shows the most commonly seen sample has two signals present. The average is slightly around 4 signals per sample and the max is 26.

In [ ]:
# For additional analysis, reinstantiate the dataset without a transform, such that the RFDescriptions can be read
datamodule = WidebandDataModule(
    root=root,
    impaired=impaired,
    qa=qa,
    fft_size=fft_size,
    num_classes=num_classes,
    transform=None,
    target_transform=None,
    batch_size=1,
    num_workers=1
)
datamodule.prepare_data()
datamodule.setup("fit")

wideband_test2 =datamodule.train

num_samples = len(wideband_test2)
snrs = []
bandwidths = []
durations = []
for idx in tqdm(range(num_samples)):
    label = wideband_test2[idx][1]
    for meta in label:
        snrs.append(meta["snr"])
        bandwidths.append(meta["bandwidth"])
        durations.append(meta["duration"])

In [ ]:
# Plot the distribution of SNR values
plt.figure(figsize=(11,4))
plt.hist(x=snrs, bins=100)
plt.title("SNR Distribution")
plt.xlabel("SNR Bins (dB)")
plt.ylabel("Counts")
plt.show()

In [ ]:
# Plot the distribution of bandwidth values
plt.figure(figsize=(11,4))
plt.hist(x=bandwidths, bins=100)
plt.title("Bandwidth Distribution")
plt.xlabel("BW Bins")
plt.ylabel("Counts")
plt.show()

In [ ]:
# Plot the distribution of bandwidth values
# plt.figure(figsize=(11,4))
# plt.hist(x=durations, bins=100)
# plt.title("Duration Distribution")
# plt.xlabel("Duration Bins")
# plt.ylabel("Counts")
# plt.show()